In [1]:
import cv2
import numpy as np
import os    

In [ ]:

# Initialize video capture
cap = cv2.VideoCapture(0)

# Load Haar cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt.xml")

# Variables
skip = 0
face_data = []
dataset_path = r'C:\Users\91878\Data Science\Face Data\\'

#avoid invalid characters in filename
import re

file_name = re.sub(r'[\\/*?:"<>|]', "", input("Enter the name of the person : "))

while True:
    ret, frame = cap.read()
    
    if not ret:
        continue

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, 1.3, 5)
    faces = sorted(faces, key=lambda f: f[2]*f[3])  # Sort by area

    for face in faces[-1:]:  # Select the largest face
        x, y, w, h = face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 255), 2)

        offset = 10
        x_start = max(x - offset, 0)
        y_start = max(y - offset, 0)
        x_end = min(x + w + offset, frame.shape[1])
        y_end = min(y + h + offset, frame.shape[0])

        face_section = frame[y_start:y_end, x_start:x_end]
        face_section = cv2.resize(face_section, (100, 100))

        skip += 1
        if skip % 10 == 0:
            face_data.append(face_section)
            print(f"Collected {len(face_data)} frames")

    # Display
    cv2.imshow("Frame", frame)
    if 'face_section' in locals():
        cv2.imshow("Face Section", face_section)

    # Break on 'q' key
    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break

# Convert and save
face_data = np.asarray(face_data)
face_data = face_data.reshape((face_data.shape[0], -1))
np.save(dataset_path + file_name + '.npy', face_data)
print(f"Data successfully saved at {dataset_path + file_name + '.npy'}")

# Cleanup
cap.release()
cv2.destroyAllWindows()


In [7]:
# -----------------KNN implementation----------
def dist(x1,x2):
    return np.sqrt(sum((x1-x2)**2))

def knn(X, Y, k=5):
    distances = []
       
    for i in range(X.shape[0]):
        ix = X[i, :-1]
        iy = X[i, -1]
        d = dist(Y, ix)
        distances.append([d,iy])
    
    #sort based on dist and get tok k
    dk = sorted(distances, key=lambda x: x[0])[:k]
    
    # Nretrieve only the labels
    labels = np.array(dk)[:, -1]
   
    #get freq of each label
    output = np.unique(labels,return_counts=True)
    
    #find max freq and corresponding label
    index = np.argmax(output[1])
    pred = output[0][index]
    return pred

In [5]:
#------------------Data Prepartion--------------

cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt.xml")

skip = 0
dataset_path = r'C:\Users\91878\Data Science\Face Data\\'

face_data = []
labels = []

class_id = 0
names ={}

#Load all face data(.npy) files
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        #create a mapping 
        names[class_id] = fx[:-4]
        print('Loaded '+fx)
        data_item = np.load(dataset_path+fx)
        face_data.append(data_item)
        
        #create lables for the class
        target = class_id*np.ones((data_item.shape[0],))
        class_id += 1
        labels.append(target)

#Stack the face data and labels
face_dataset = np.concatenate(face_data, axis=0)
face_labels = np.concatenate(labels, axis=0).reshape((-1,1))
print(face_dataset.shape)
print(face_labels.shape)

trainset = np.concatenate((face_dataset, face_labels), axis=1)
print("Taining data shape", trainset.shape)

# --------Real-time testing----------

while True:
    ret,frame = cap.read()
    
    if ret == False:
        continue
    faces = face_cascade.detectMultiScale(frame,1.3,5)
  
    for face in faces:
        x,y,w,h = face
        
        offset = 10
        x_start = max(x - offset, 0)
        y_start = max(y - offset, 0)
        x_end = min(x + w + offset, frame.shape[1])
        y_end = min(y + h + offset, frame.shape[0])
        
        face_section = frame[y_start:y_end, x_start:x_end]
        if face_section.size == 0:
            continue
        
        face_section = cv2.resize(face_section,(100,100))    
        
        #predicated label out
        out = knn(trainset, face_section.flatten())
        
        pred_name = names[int(out)]
        cv2.putText(frame, pred_name, (x,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h), (0,255,255),2)
        
    cv2.imshow('Face Recognition',frame)
    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()        

Loaded ravi.npy
(3, 30000)
(3, 1)
(3, 30001)
